# 神经网络的引擎: 基于梯度的优化

正如上一节看到的,第一个例子中每个密集层对输入数据做了如下变换

```py
output = relu(dot(input, W) + b)
```

上式中,`W` 和 `b` 是张量,属于该层的属性.被称为该层的权重/训练参数.神经网络从训练数据的信息就储存在这些权重中.

最初这些权重矩阵被随机值填充,当然不能期望初始状态下随机的权重能有什么效果.但是这是一个起点,接下来就是根据反馈信号不断调整这些权重,这个过程就是训练.

上面的流程称之为训练循环,在一个循环中不断重复,直到损失值足够低.

1. 输入一批训练样本 `x`,以及对应的标签 `y_true`.
2. 在模型上运行 `x` 获取预测值 `y_pred`.(这个过程称为前向传播)
3. 计算 `y_pred` 和 `y_true` 的损失值 `loss`,这是衡量 `y_pred` 和 `y_true` 不匹配程度的标准.
4. 更新模型的全部权重,以略微降低损失值.

最终我们会得到一个在训练数据上损失值很低的模型.这个模型已经学会了见训练数据正确的映射到目标上.但是整个过程是如何完成的?其实一步一步拆解来看,整个流程非常简单.

第一步仅仅是输入输出操作,没什么神秘的.第二部和第三步是少量张量操作.困难的地方在第四步: 更新权重.如何更新单个权重?增加还是减少?与其他权重的关系?

一个最直接的办法: 每次只更新一个权重,把单个权重调整到最优值,再重复直到更新完全部的权重.

直接说结论: 这样的方式效率非常低,一个典型的网络可能存在上千甚至百万级别的系数,计算的代价很大.我们还有更好的办法: 梯度下降.


梯度下降的基本思想是: 

- 我们在模型中使用的函数都是以平滑和连续的方式转换其输入(例如z=x+y,y的微小变化会导致z的微小变化,通过y变化的方向就能推测z变化的方向).数学上称这个性质为可微.
- 可微还有一个特性是: 将可微的函数组合在一起,依旧是可微的的.
- 这使得我们可以通过一个名为梯度的算子来推测权重矩阵想不同方向变化时,损失可能的变化方向.这样通过计算梯度我们就能让权重向损失值减小的方向移动,注意: 这里是可以一次性更新全部的权重而不是单独某一个权重.
